In [94]:
import numpy as np
import pandas as pd
from Sector_Exposure import Rate_Exposures
import fredapi as fa
from datetime import date
import Useful_Functions as u
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

fred = fa.Fred('4fb0ce271d0f66f4b5b3904b4aaf1dd0')

cutoff = .7

sectors_df = pd.read_csv("sectors_and_rates.csv", index_col = "Date")

train_df = sectors_df[:int(len(sectors_df)*cutoff)]
test_df = sectors_df[int(len(sectors_df)*cutoff):]
#sectors_df['inflation'] = fred.get_series('T10YIE', observation_start = '2010-01-04', end = date.today())


In [95]:

# , "inflation"
train_exposure = Rate_Exposures(train_df, 252, 30, ['Real Yield', 'Yield Curve'], "SP-500")
test_exposure = Rate_Exposures(test_df, 252, 30, ['Real Yield', 'Yield Curve'], "SP-500")

In [96]:

features = train_exposure.monthly_scores[u.ry]
labels = [int(i) for i in train_exposure.forward_returns_monthly[u.ut] > 0.0]
train_frame = pd.DataFrame()
train_frame['above_market'] = pd.Series(labels, index = train_exposure.forward_returns_monthly.index)
train_frame['ry_scores'] = features
train_frame = train_frame.dropna()



In [97]:

labels_test = [int(i) for i in test_exposure.forward_returns_monthly[u.ut] > 0.0]
test_frame = pd.DataFrame()
test_frame['above_market'] = pd.Series(labels_test, index = test_exposure.forward_returns_monthly.index)
test_frame['ry_scores'] = test_exposure.monthly_scores[u.ry]
test_frame = test_frame.dropna()

X = np.array(train_frame['ry_scores']).reshape(-1,1)
y = train_frame['above_market']

X_test = np.array(test_frame['ry_scores']).reshape(-1,1)
y_test = test_frame['above_market']

forest = RandomForestClassifier().fit(X, y)

preds = forest.predict(X_test)

accuracy_score(y_test, preds)


0.4482758620689655

In [99]:

evaluate = pd.DataFrame(index = test_frame.index)
evaluate['above_market'] = X_test
evaluate['returns'] = test_exposure.forward_returns_monthly[u.ut]
evaluate['preds'] = preds

evaluate.returns.loc[evaluate.preds == 1].mean()


-0.016796926390312068